Sources:
    https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb
    https://stackoverflow.com/questions/55204998/crop-and-select-only-the-detected-region-from-an-image-in-python
    
Edited by Sebastian Gampe

In [2]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
    raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')

In [2]:
# This is needed to display the images.
%matplotlib inline


from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

In [3]:
# What model to download. ist alles schon unter Volume vorhanden, paths ändern!
#MODEL_NAME = 'faster_rcnn_resnet101_coco_2018_01_28'
#MODEL_FILE = MODEL_NAME + '.tar.gz'
#DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to model
PATH_TO_MODEL = '/home/user/.local/lib/python3.6/site-packages/tensorflow/models/research/object_detection/models/faster_rcnn_resnet101_coco_2018_01_28'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = '/home/user/portrait_separator/model/' + 'frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = '/home/user/portrait_separator/coin_label_map.pbtxt'

In [4]:
# Only neccessary if you need the original model for a reason
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
    file_name = os.path.basename(file.name)
    if 'frozen_inference_graph.pb' in file_name:
        tar_file.extract(file, os.getcwd())

In [4]:
# load the frozen graph
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [5]:
# labels
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# image convert
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

In [32]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
#PATH_TO_TEST_IMAGES_DIR = '/opt/conda/lib/python3.6/site-packages/object_detection/test_images'
PATH_TO_TEST_IMAGES_DIR = '/home/user/ML/Data/CoinPortraitData/train/augustus'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'augustus{}.png'.format(i)) for i in range(704, 820) ]

PATH_TO_OUTPUT = '/home/user/ML/Data/CoinPortraitData/output'
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [6]:
# Inference function
def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                    tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[1], image.shape[2])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                    feed_dict={image_tensor: image})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.int64)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

In [7]:
# Seperates the inside of the box the rest of the image
def seperate_box_from_image(image_np, output_dict):
    img_height, img_width, img_channel = image_np.shape
    absolute_coord = []
    THRESHOLD = 0.7 # adjust your threshold here
    N = len(output_dict['detection_boxes'])
    for i in range(N):
        if output_dict['detection_scores'][i] < THRESHOLD:
            continue
        box = output_dict['detection_boxes'][i]
        ymin, xmin, ymax, xmax = box
        x_up = int(xmin*img_width)
        y_up = int(ymin*img_height)
        x_down = int(xmax*img_width)
        y_down = int(ymax*img_height)
        absolute_coord.append((x_up,y_up,x_down,y_down))
    
    bounding_box_img = []
    for c in absolute_coord:
        bounding_box_img.append(image_np[c[1]:c[3], c[0]:c[2],:])
    return bounding_box_img 

In [20]:
data_dir = '/home/user/ML/Data/CoinPortraitData/train/'
output_path = '/home/user/ML/Data/CoinPortraitData/output'

# List all image files (.jpg or .png or .tif) in subfolders of dir
def list_files(data_dir):
    r = []                                                                                                                                                                                                      
    for dirpath, subdir, files in os.walk(data_dir):
        for file in files:
            if (file.endswith(".jpg")) or (file.endswith(".png")) or (file.endswith(".tif")): 
                r.append(os.path.join(dirpath, file)) 
    #for k in range(10):
        #print(r[k])
    return r 

files = list_files(data_dir)
files.sort()
IMAGE_SIZE = (12, 8)

In [21]:
# Uses the inference function on every picture in the given path
for image_path in files:
    try:
        image = Image.open(image_path)
    except OSError:
        continue
    
    # make folders for the boxed images
    pathWords = image_path.split('/')
    emperor = pathWords[len(pathWords)-2]
    file_name = pathWords[len(pathWords)-1]
    file_name = file_name.replace('.png','')
    emperor_output = output_path + '/' + emperor +'/'
    emperor_output_cropped =  emperor_output + '/cropped/'
    
    if not os.path.exists(emperor_output):
        os.makedirs(emperor_output)
        os.makedirs(emperor_output_cropped)
        
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    try:
        image_np = load_image_into_numpy_array(image)
    except ValueError:
        continue
        
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
    # Crop the contents of the bounding box
    image_np2 = seperate_box_from_image(image_np, output_dict)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=0)
    plt.figure(figsize=IMAGE_SIZE)
    #plt.imshow(image_np)
    
    # save the images
    try:
        im2 = Image.fromarray(image_np)
        #im2.save(emperor_output + emperor + str(i) + '.jpg')
        im2.save(emperor_output + file_name + '.jpg')
        #plt.savefig(PATH_TO_OUTPUT + '/augustus/augustus' + str(i) + '.jpg')
        im = Image.fromarray(image_np2[0])
        #im.save(emperor_output_cropped + emperor + '_cr' + str(i) + '.jpg')
        im.save(emperor_output_cropped + file_name + '.jpg')
        
    except IndexError:
        continue
    except ValueError:
        continue
        
    plt.close('all')
        
plt.show()
print(str(i) + '.jpg')
plt.close()

5.jpg
